# Lab 8: Audio Classification 

## Part 1: Making a speech detector

In this section we will design a simple classifier that will let us know if its input is speech or non-speech. Download the data archive from: [ https://drive.google.com/file/d/1oAnvk-hzzgzZ4di4W0pKw6v3IWLm9u2X/view?usp=sharing ] In this part we will use the dataset in data/SpeechMusic. In it you will find two directories, speech/ and music/ containing data from each class.

Randomly select 50 soundfiles from each directory to use as training data, and use the remaining sounds as testing data. For all of the sounds we will compute a representation that makes the classification easier and we will use a simple Gaussian model to classify them. Do the following:

- Perform an STFT for each sound, take it’s magnitude and raise it to 0.3 to improve contrast
    - We will consider each spectral slice of that to be a data point
- Using the training data of each sound:
    - Calculate the mean column and the diagonal covariance of the columns
    - You will thus get two sets of Gaussian parameters that model each sound class
- For each testing data point:
    - Calculate the likelihood of each column based on the above models
	- To calculate the entire file likelihood add all the frame likelihoods
	- Assign each soundfile to the class that gets the highest likelihood

For extra credit implement the parameter estimation and model likelihood yourself. If you are too lazy for that you can instead use ```sklearn.mixture.GaussianMixture``` to learn a diagonal single-Gaussian model per class.

How do the results look like? If you rerun this with a different training/testing set, is there an appreciable difference? On average over multiple training/testing sets what accuracy do you get?

In [72]:
# YOUR CODE HERE
import numpy as np
import math
import IPython
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
import scipy.signal as sg
from tqdm import tqdm
from os import listdir
from sklearn.model_selection import train_test_split 
from sklearn.mixture import GaussianMixture

def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    display( HTML( 
    '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
    '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
    ))

def stft( input_sound, dft_size, hop_size, zero_pad, window):
    # YOUR CODE HERE
    padZeroes = np.zeros(dft_size)
    pad = np.append(padZeroes, input_sound)
    pad = np.append(pad, padZeroes)
    tempMatrix = []
    for i in range(0, len(pad)-dft_size, hop_size):
        tempMatrix.append(pad[i:i+dft_size] * window)
    out = []
    for i in range(len(tempMatrix)):
        rfft = np.fft.rfft(tempMatrix[i], dft_size+zero_pad)
        out.append(np.reshape(rfft, (-1, 1)))     
    return np.hstack(out)

def istft( stft_output, dft_size, hop_size, zero_pad, window):
    # YOUR CODE HERE
    irfftSounds = []
    for segment in stft_output.T:
        irfftSounds.append(np.fft.irfft(segment, dft_size+zero_pad))
    irfftSounds = np.array(irfftSounds)
    input_sound = np.zeros(dft_size+hop_size*(len(irfftSounds)-1))
    for i in range(len(irfftSounds)):
        input_sound[i*hop_size:i*hop_size+dft_size] += (irfftSounds[i][:dft_size] * window)
    input_sound = input_sound[dft_size:]
    return input_sound

def loadDir(directory):
    audioData = []
    rate = 0
    for filename in tqdm(listdir(directory)):
        if filename.endswith(".wav"):
            curFilePath = os.path.join(directory, filename)
            #rate should be same for all files, only 1 variable needed
            wavRate, wavData = wavfile.read(curFilePath)
            audioData.append(wavData)
            if rate != wavRate:
                rate = wavRate
    return audioData, rate


def getSTFT(dataList):
    stftList = []
    for data in tqdm(dataList):
        curSTFT = stft(data,  dft_size, hop_size, zero_pad, window)
        magnitude = np.abs(curSTFT)
        curSTFT = np.power(magnitude, 0.3)
        stftList.append(curSTFT)
    return stftList


#computer performance method should work, as it is from a past project of mine, 
#but the numbers it gives me are weird for accuracy, so I probably implemented it wrong

# def computePerformance(predictedLabels, actualLabels):
#     yhats = predictedLabels
#     accuracy = np.mean(yhats == actualLabels)
#     tp = np.sum([yhats[i] == actualLabels[i] and yhats[i] == 1 for i in range(len(yhats))])
#     precision = tp / np.sum([yhats[i] == 1 for i in range(len(yhats))])
#     recall = tp / (np.sum([yhats[i] != actualLabels[i] and yhats[i] == 0 for i in range(len(yhats))]) + tp)
#     f1 = 2 * (precision * recall) / (precision + recall)
#     return accuracy, f1, precision, recall

def getPerformance(gaussianSpeech, gaussianMusic, speechTest, musicTest):
    predictedLabels = []
    actualLabels = []
    correct = 0
    #assume 1 is speech, 0 is music
    for speech in speechTest:
        scoreSpeech = gaussianSpeech.score(speech)
        scoreMusic = gaussianMusic.score(speech)
        if scoreSpeech > scoreMusic:
            correct += 1
            predictedLabels.append(1) #speech
        else:
            predictedLabels.append(0) #music
        actualLabels.append(1)
    for music in musicTest:
        scoreSpeech = gaussianSpeech.score(music)
        scoreMusic = gaussianMusic.score(music)
        if scoreSpeech < scoreMusic:
            correct += 1
            predictedLabels.append(0) #music
        else:
            predictedLabels.append(1) #speech
        actualLabels.append(0)
    print(predictedLabels)
    print(actualLabels)
    accuracy = np.mean(predictedLabels == actualLabels)
    testSize = len(speechTest) + len(musicTest)
    accuracy2 = correct/testSize
    print("accuracy ", accuracy)
    print("accuracy2 ", accuracy2)
#     accuracy, f1, precision, recall = computePerformance(predictedLabels, actualLabels)
#     return accuracy, f1, precision, recall
    return accuracy
# raise NotImplementedError()

In [73]:
dft_size = 1024
hop_size = 256
zero_pad = 0
window = sg.hann(dft_size)

dataDir = "data/SpeechMusic/"
musicDir = dataDir+"music/"
speechDir = dataDir+"speech/"



def runTrial(musicDir, speechDir, dft_size, hop_size, zero_pad, window, random = True):
    print("loading files")
    audioDataMusic, rateMusic = loadDir(musicDir)
    audioDataSpeech, rateSpeech = loadDir(speechDir)
    print("--- loading files complete ---")

    print("performing STFT")
    stftMusic = getSTFT(audioDataMusic)
    stftSpeech = getSTFT(audioDataSpeech)
    print("--- STFT complete ---")


    print("split data")
    if random:
        speechTrain, speechTest, musicTrain, musicTest = train_test_split( stftSpeech, stftMusic, train_size=50, test_size=10)
    else:
        speechTrain, speechTest, musicTrain, musicTest = train_test_split( stftSpeech, stftMusic, train_size=50, test_size=10, random_state=32)
    speechTrain = np.concatenate(np.array(speechTrain), axis=0)
    musicTrain = np.concatenate(np.array(musicTrain), axis=0)
    print(len(speechTrain))
    print(len(speechTest))
    print(len(musicTrain))
    print(len(musicTest))
    print("---- data split complete ---- ")


    #create and fit Gaussian models
    print("fit Speech model")
    gaussianSpeech = GaussianMixture(5, "diag",verbose=1)
    gaussianSpeech = gaussianSpeech.fit(speechTrain)
    print("---- Speech model fitted ---- ")

    print("fit Music model")
    gaussianMusic = GaussianMixture(5, "diag",verbose=1)
    gaussianMusic = gaussianMusic.fit(musicTrain)
    print("---- Music model fitted ---- ")



    #get performance
    print("calculating performance")
#     accuracy, f1, precision, recall = getPerformance(gaussianSpeech, gaussianMusic, speechTest, musicTest)
    accuracy = getPerformance(gaussianSpeech, gaussianMusic, speechTest, musicTest)
    print("---- performance calculated -----")

    print("results:")
    print("accuracy: ", accuracy)
    print("f1: ", f1)
    print("precision: ", precision)
    print("recall: ", recall)


In [74]:
#this is an example run, because

runTrial(musicDir, speechDir, dft_size, hop_size, zero_pad, window,random=False)

loading files


100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 1621.61it/s]


--- loading files complete ---
performing STFT


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:03<00:00, 15.64it/s]


--- STFT complete ---
split data
25650
10
25650
10
---- data split complete ---- 
fit Speech model


C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass covariance_type=diag as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
Initialization converged: True
---- Speech model fitted ---- 
fit Music model


C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass covariance_type=diag as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
Initialization converged: True
---- Music model fitted ---- 
calculating performance
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
accuracy  0.0
accuracy2  0.9
---- performance calculated -----
results:
accuracy:  0.0


NameError: name 'f1' is not defined

In [61]:
#warning, I recommend commenting out most of the prints if you want to run a bunch of trials, because it gets messy otherwise
#sorry

for i in range(10):
    print("\n\n\n--- Trial Number :", i,"--- ")
    runTrial(musicDir, speechDir, dft_size, hop_size, zero_pad, window)

loading files


100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 1935.56it/s]


--- loading files complete ---
performing STFT


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:03<00:00, 15.78it/s]


--- STFT complete ---
split data
---- data split complete ---- 
fit Speech model


...

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass covariance_type=diag as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
Initialization converged: True
---- Speech model fitted ---- 
fit Music model


C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass covariance_type=diag as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Initialization 0


KeyboardInterrupt: 

## after running multiple trials, I found that accuracy varies from

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass covariance_type=diag as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
Initialization converged: True


C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass covariance_type=diag as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
Initialization converged: True


In [55]:
print("Correct:", correct)

Correct: 19 Test Size: 20 Accuracy: 0.95


## Part 2: Making a music genre classifier

We will repeat the above, but this time we will perform music genre classification. To do so we will use a slightly more elaborate feature representation, and a stronger classification model. If you downloaded the data archive pointed to above, you will find a subset of the CTZAN dataset in the data/genre folder, this is a benchmark data set for music genre classification.

Just as before, you will find a set of directories with examples of each sound class that we want to recognize. For each class, split the soundfiles into a training set (50% of data) and testing set (remaining 50% of data).

For a representation we will use MFCC features. For extra credit, code these yourself otherwise you can use the implementation from the ```librosa``` library. Once all the files are transformed we will have a series of MFCC frames for each recording (as opposed to spectral frames as is in the case of the STFT). We will use these as the data to classify.

For each class learn a Gaussian model (with a diagonal covariance again). This will be the same process as above.
In order to evaluate how good this works we will use the following procedure. For each sound in the training data, get the likelihood of each MFCC frame based on the learned Gaussian models and sum these over the entire file just as we did before. Use the resulting values to get a classification result for each . Report how accurate your results are. Now report the accuracy using your testing data instead.

Now will use a better classifier to hopefully get better accuracy. We will use a Gaussian Mixture Model (```sklearn.mixture.GaussianMixture```). Just as before you should learn one such model for each class using the corresponding training data.

How many Gaussians do you need in your GMM to get the best results? Do the MFCC parameters make a difference? Play around with the numbers to get the best possible results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part 3: Make it better (extra credit, required for 4-hour registrants)

There is no shortage of techniques (and free code) to use for classification. Revisit the two problems above and use any other type of classifier you want (Neural Nets, Boosting, Decision Trees, whatever). Also feel free to use any feature you want. Can you improve on the results you got before? How much higher can you get your accuracy for either case?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()